In [1]:
#Import Dependencies
import pandas as pd
import sqlite3
from sqlite3 import Error

In [2]:
# Edit display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Connection to sqlite db
con = sqlite3.connect('./sqlite_db.sqlite')
cursor = con.cursor()

In [4]:
# Query db w/ joins to get full stat tables
positions = ['qb', 'rb', 'te', 'wr']
df_list = []
for x in positions:
    df = f'{x}_df'
    table = f'{x.upper()}_Stats'
    table2 = f'{x.upper()}_Extra_Stats'
    df = pd.DataFrame(pd.read_sql_query(f'select * from {table} as a inner join {table2} as b on a.PlayerID = b.PlayerPlayerId and a.Year=b.Year', con))
    df_list.append(df)

In [5]:
#Close Connection to sqlite db
# con.close()

In [6]:
#Check for loop output
print(len(df_list))
print(type(df_list[1]))

4
<class 'pandas.core.frame.DataFrame'>


In [7]:
#Create empty dbs to merge from list
qb_df = pd.DataFrame()
rb_df = pd.DataFrame()
te_df = pd.DataFrame()
wr_df = pd.DataFrame()

In [8]:
#Merge dataframes from list
qb_df = pd.concat([df_list[0], qb_df], ignore_index=True)
rb_df = pd.concat([df_list[1], rb_df], ignore_index=True)
te_df = pd.concat([df_list[2], te_df], ignore_index=True)
wr_df = pd.concat([df_list[3], wr_df], ignore_index=True)


In [9]:
#Quick check on QB DF
print(qb_df.head(1))
print(qb_df.columns)

   index  Rank  PlayerID        Name Team Position  Played  \
0      0     1     19801  Josh Allen  BUF       QB      17   

   PassingCompletions  PassingAttempts  PassingCompletionPercentage  \
0                 409              646                         63.3   

   PassingYards  PassingYardsPerAttempt  PassingTouchdowns  \
0          4407                     6.8                 36   

   PassingInterceptions  PassingRating  RushingAttempts  RushingYards  \
0                    15          92.17              122           763   

   RushingYardsPerAttempt  RushingTouchdowns  FantasyPointsPerGame  \
0                     6.3                  6                  23.7   

   FantasyPoints  Year  index  Rank  PlayerPlayerId PlayerShortName  \
0         402.58  2021      0     1           19801        J. Allen   

   PlayerAgeExact  PassAttemptsPerGame  PassingYards  CompletionPercentage  \
0            26.1                 37.9          4407                 63.41   

   PassingTouchdown

In [10]:
#Quick check on RB DF
print(rb_df.head(1))
print(rb_df.columns)

   index  Rank  PlayerID             Name Team Position  Played  \
0      0     1     21682  Jonathan Taylor  IND       RB      17   

   RushingAttempts  RushingYards  RushingYardsPerAttempt  RushingTouchdowns  \
0              332          1811                     5.5                 18   

   ReceivingTargets  Receptions  ReceivingYards  ReceivingTouchdowns  Fumbles  \
0                51          40             360                    2        4   

   FumblesLost  FantasyPointsPerGame  FantasyPoints  Year  index  Rank  \
0            2                  19.6          333.1  2021      0     1   

   PlayerPlayerId PlayerShortName  PlayerAgeExact  Carries  RushingYards  \
0           21682       J. Taylor            23.5      332          1811   

   YardsPerCarry  RushingTouchdowns  Targets  Receptions  ReceivingYards  \
0           5.45                 18       53          40             360   

   ReceivingTDs  SnapShare  OpportunityShare  ShotgunCarryRate  \
0             2      6

In [11]:
#Quick check on WR DF
print(wr_df.head(1))
print(wr_df.columns)

   index  Rank  PlayerID         Name Team Position  Played  ReceivingTargets  \
0      0     1     18882  Cooper Kupp  LAR       WR      17               191   

   Receptions  ReceptionPercentage  ReceivingYards  ReceivingTouchdowns  \
0         145                 75.9            1947                   16   

   ReceivingLong  ReceivingYardsPerTarget  ReceivingYardsPerReception  \
0             59                     10.2                        13.4   

   RushingAttempts  RushingYards  RushingYardsPerAttempt  RushingTouchdowns  \
0                4            18                     4.5                  0   

   Fumbles  FumblesLost  FantasyPointsPerGame  FantasyPoints  Year  index  \
0        0            0                  17.3          294.5  2021      0   

   Rank  PlayerPlayerId PlayerShortName  Targets  Receptions  ReceivingYards  \
0     1           18882         C. Kupp      191         145            1947   

   ReceivingTDs  HogRate  SnapShare  AverageTargetDistance  AirY

In [12]:
#Quick check on TE DF
print(te_df.head(1))
print(te_df.columns)

   index  Rank  PlayerID          Name Team Position  Played  \
0      0     1     19803  Mark Andrews  BAL       TE      17   

   ReceivingTargets  Receptions  ReceptionPercentage  ReceivingYards  \
0               154         107                 69.5            1361   

   ReceivingTouchdowns  ReceivingLong  ReceivingYardsPerTarget  \
0                    9             43                      8.8   

   ReceivingYardsPerReception  RushingAttempts  RushingYards  \
0                        12.7                1             0   

   RushingYardsPerAttempt  RushingTouchdowns  Fumbles  FumblesLost  \
0                     0.0                  0        1            0   

   FantasyPointsPerGame  FantasyPoints  Year  index  Rank  PlayerPlayerId  \
0                  11.4          194.1  2021      0     1           19803   

  PlayerShortName  Targets  Receptions  ReceivingYards  ReceivingTDs  HogRate  \
0      M. Andrews      154         107            1361             9      NaN   

   Sn

In [13]:
''' 
Identify duplicate columns 
Code used from this reference:
https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/
'''
def getDuplicateColumns(df):
  
    # Create an empty set
    duplicateColumnNames = set()
      
    # Iterate through all the columns 
    # of dataframe
    for x in range(df.shape[1]):
          
        # Take column at xth index.
        col = df.iloc[:, x]
          
        # Iterate through all the columns in
        # DataFrame from (x + 1)th index to
        # last index
        for y in range(x + 1, df.shape[1]):
              
            # Take column at yth index.
            otherCol = df.iloc[:, y]
               # Check if two columns at x & y
            # index are equal or not,
            # if equal then adding 
            # to the set
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
                  
    # Return list of unique column names 
    # whose contents are duplicates.
    return list(duplicateColumnNames)

In [14]:
# Set dup column array names (organizational purposes)
dupColumnsNames = []
for x in range(0, len(positions)):
    col_name = f'dupColumns_{positions[x]}'
    dupColumnsNames.append(col_name)
dupColumnsNames

['dupColumns_qb', 'dupColumns_rb', 'dupColumns_te', 'dupColumns_wr']

In [15]:
# get columns w/ duplicate values for each dataframe

dupColumns = []
for i in range(0, len(dupColumnsNames)):
    dupColumns.append(getDuplicateColumns(df_list[i]))
dupColumns

[['Interceptions', 'PassingTouchdowns', 'Year', 'PlayerPlayerId'],
 ['Year', 'PlayerPlayerId', 'Receptions'],
 ['ReceivingTDs', 'Year', 'PlayerPlayerId', 'Receptions', 'Targets.1'],
 ['Targets.1', 'Year', 'PlayerPlayerId', 'Receptions']]

In [16]:
#Check column names
for frame in df_list:
    print(frame.columns)


Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'PassingCompletions', 'PassingAttempts', 'PassingCompletionPercentage',
       'PassingYards', 'PassingYardsPerAttempt', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGame', 'FantasyPoints', 'Year', 'index', 'Rank',
       'PlayerPlayerId', 'PlayerShortName', 'PlayerAgeExact',
       'PassAttemptsPerGame', 'PassingYards', 'CompletionPercentage',
       'PassingTouchdowns', 'Interceptions', 'MoneyThrows', 'TouchdownRate',
       'ProtectionRate', 'PressuredCompletionPercentage', 'AirYards',
       'AirYardsPerGame', 'AirYardsPerAttempt', 'DeepBallAttempts',
       'DeepBallCompletionPercentage', 'Year'],
      dtype='object')
Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       '

In [17]:
#Check count of columns per df
i = 0 # mini index
for frame in df_list:
    print(f'{positions[i]} df: {len(frame.columns)}')
    i += 1

qb df: 42
rb df: 44
te df: 45
wr df: 45


In [18]:
# Create new list of df dropping duplicate columns with same name
df_list2 = []
for frame in df_list:
    frame2 = frame.loc[:,~frame.columns.duplicated()].copy()
    df_list2.append(frame2)

In [19]:
#Column count after duplicates dropped
i = 0 # mini index
for frame in df_list2:
    print(f'{positions[i]} df: {len(frame.columns)}')
    i += 1

qb df: 37
rb df: 37
te df: 40
wr df: 40


In [20]:
# Check column names after drops
for frame in df_list2:
    print(frame.columns)

Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'PassingCompletions', 'PassingAttempts', 'PassingCompletionPercentage',
       'PassingYards', 'PassingYardsPerAttempt', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGame', 'FantasyPoints', 'Year', 'PlayerPlayerId',
       'PlayerShortName', 'PlayerAgeExact', 'PassAttemptsPerGame',
       'CompletionPercentage', 'Interceptions', 'MoneyThrows', 'TouchdownRate',
       'ProtectionRate', 'PressuredCompletionPercentage', 'AirYards',
       'AirYardsPerGame', 'AirYardsPerAttempt', 'DeepBallAttempts',
       'DeepBallCompletionPercentage'],
      dtype='object')
Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'ReceivingTargets', 'Receptions', 'ReceivingYards

In [21]:
#Check to see if duplicate values still exist- second time
dupColumns2 = []
for i in range(0, len(dupColumnsNames)):
    dupColumns2.append(getDuplicateColumns(df_list2[i]))
dupColumns2

[['Interceptions', 'PlayerPlayerId'],
 ['PlayerPlayerId'],
 ['ReceivingTDs', 'PlayerPlayerId', 'Targets.1'],
 ['PlayerPlayerId', 'Targets.1']]

In [22]:
#Drop colunmns in dup values list... columns that contain same values as another column w/ different column name
for i in range(0, len(df_list2)):
    df_list2[i].drop(columns = dupColumns2[i], axis = 1, inplace = True)

In [23]:
#Print colun names for check
for frame in df_list2:
    print(frame.columns)

Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'PassingCompletions', 'PassingAttempts', 'PassingCompletionPercentage',
       'PassingYards', 'PassingYardsPerAttempt', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGame', 'FantasyPoints', 'Year', 'PlayerShortName',
       'PlayerAgeExact', 'PassAttemptsPerGame', 'CompletionPercentage',
       'MoneyThrows', 'TouchdownRate', 'ProtectionRate',
       'PressuredCompletionPercentage', 'AirYards', 'AirYardsPerGame',
       'AirYardsPerAttempt', 'DeepBallAttempts',
       'DeepBallCompletionPercentage'],
      dtype='object')
Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'ReceivingTargets', 'Receptions', 'ReceivingYards',
       'ReceivingTouchdowns', 'F

In [24]:
#Column count after duplicates dropped
i = 0 # mini index
for frame in df_list2:
    print(f'{positions[i]} df: {len(frame.columns)}')
    i += 1

qb df: 35
rb df: 36
te df: 37
wr df: 38


In [25]:
#Drop column named Index. 
for frame in df_list2:
    frame.drop(columns = 'index', axis = 1, inplace= True)

In [26]:
#identify nulls in the data
for frame in df_list2:
    print(frame.isnull().sum())

#QBS - none, WR / TE - Hog Rate, RB - a good amount

Rank                             0
PlayerID                         0
Name                             0
Team                             0
Position                         0
Played                           0
PassingCompletions               0
PassingAttempts                  0
PassingCompletionPercentage      0
PassingYards                     0
PassingYardsPerAttempt           0
PassingTouchdowns                0
PassingInterceptions             0
PassingRating                    0
RushingAttempts                  0
RushingYards                     0
RushingYardsPerAttempt           0
RushingTouchdowns                0
FantasyPointsPerGame             0
FantasyPoints                    0
Year                             0
PlayerShortName                  0
PlayerAgeExact                   0
PassAttemptsPerGame              0
CompletionPercentage             0
MoneyThrows                      0
TouchdownRate                    0
ProtectionRate                   0
PressuredCompletionP

In [27]:
#Quick look at the data
for frame in df_list2:
    print(frame.head(1))

   Rank  PlayerID        Name Team Position  Played  PassingCompletions  \
0     1     19801  Josh Allen  BUF       QB      17                 409   

   PassingAttempts  PassingCompletionPercentage  PassingYards  \
0              646                         63.3          4407   

   PassingYardsPerAttempt  PassingTouchdowns  PassingInterceptions  \
0                     6.8                 36                    15   

   PassingRating  RushingAttempts  RushingYards  RushingYardsPerAttempt  \
0          92.17              122           763                     6.3   

   RushingTouchdowns  FantasyPointsPerGame  FantasyPoints  Year  \
0                  6                  23.7         402.58  2021   

  PlayerShortName  PlayerAgeExact  PassAttemptsPerGame  CompletionPercentage  \
0        J. Allen            26.1                 37.9                 63.41   

   MoneyThrows  TouchdownRate  ProtectionRate  PressuredCompletionPercentage  \
0           29           5.58           77.94     

In [28]:
#Split positions out of list and into individual dfs
qb_full_stats = df_list2[0]
rb_full_stats = df_list2[1]
te_full_stats = df_list2[2]
wr_full_stats = df_list2[3]

In [29]:
#Drop column w/ Nans for wr and te
wr_full_stats.drop(columns='HogRate', axis = 1, inplace=True)
te_full_stats.drop(columns='HogRate', axis = 1, inplace=True)

In [30]:
#Check to ensure no more nans
wr_full_stats.isnull().sum()

Rank                          0
PlayerID                      0
Name                          0
Team                          0
Position                      0
Played                        0
ReceivingTargets              0
Receptions                    0
ReceptionPercentage           0
ReceivingYards                0
ReceivingTouchdowns           0
ReceivingLong                 0
ReceivingYardsPerTarget       0
ReceivingYardsPerReception    0
RushingAttempts               0
RushingYards                  0
RushingYardsPerAttempt        0
RushingTouchdowns             0
Fumbles                       0
FumblesLost                   0
FantasyPointsPerGame          0
FantasyPoints                 0
Year                          0
PlayerShortName               0
Targets                       0
ReceivingTDs                  0
SnapShare                     0
AverageTargetDistance         0
AirYards                      0
AirYardsPerGame               0
AirYardsPerReception          0
RedZoneT

In [31]:
#Check to ensure no more nans
te_full_stats.isnull().sum()

Rank                          0
PlayerID                      0
Name                          0
Team                          0
Position                      0
Played                        0
ReceivingTargets              0
Receptions                    0
ReceptionPercentage           0
ReceivingYards                0
ReceivingTouchdowns           0
ReceivingLong                 0
ReceivingYardsPerTarget       0
ReceivingYardsPerReception    0
RushingAttempts               0
RushingYards                  0
RushingYardsPerAttempt        0
RushingTouchdowns             0
Fumbles                       0
FumblesLost                   0
FantasyPointsPerGame          0
FantasyPoints                 0
Year                          0
PlayerShortName               0
Targets                       0
SnapShare                     0
AverageTargetDistance         0
AirYards                      0
AirYardsPerGame               0
AirYardsPerReception          0
RedZoneTargets                0
EndzoneT

In [32]:
#Look at RB stats
rb_full_stats.isnull().sum()

Rank                         0
PlayerID                     0
Name                         0
Team                         0
Position                     0
Played                       0
RushingAttempts              0
RushingYards                 0
RushingYardsPerAttempt       0
RushingTouchdowns            0
ReceivingTargets             0
Receptions                   0
ReceivingYards               0
ReceivingTouchdowns          0
Fumbles                      0
FumblesLost                  0
FantasyPointsPerGame         0
FantasyPoints                0
Year                         0
PlayerShortName              0
PlayerAgeExact               0
Carries                      0
YardsPerCarry               30
Targets                      0
ReceivingTDs                 0
SnapShare                    0
OpportunityShare             6
ShotgunCarryRate            58
ShotgunYardsPerCarry        67
UnderCenterCarryRate        58
UnderCenterYardsPerCarry    40
AverageDefendersInTheBox    58
StackedF

In [33]:
#Filter to RB that have 0 carries to look at missing values
rb_full_stats[rb_full_stats["Carries"] == 0].count()

Rank                        52
PlayerID                    52
Name                        52
Team                        52
Position                    52
Played                      52
RushingAttempts             52
RushingYards                52
RushingYardsPerAttempt      52
RushingTouchdowns           52
ReceivingTargets            52
Receptions                  52
ReceivingYards              52
ReceivingTouchdowns         52
Fumbles                     52
FumblesLost                 52
FantasyPointsPerGame        52
FantasyPoints               52
Year                        52
PlayerShortName             52
PlayerAgeExact              52
Carries                     52
YardsPerCarry               28
Targets                     52
ReceivingTDs                52
SnapShare                   52
OpportunityShare            52
ShotgunCarryRate             0
ShotgunYardsPerCarry        28
UnderCenterCarryRate         0
UnderCenterYardsPerCarry    28
AverageDefendersInTheBox     0
StackedF

In [34]:
#Fill all nans with 0 since RB had 0 carries
rb_full_stats[rb_full_stats["Carries"] == 0] = rb_full_stats[rb_full_stats["Carries"] == 0].fillna(value = 0)

In [35]:
#Check to make sure all nulls are filled w/ RBs with 0 carries
rb_full_stats[rb_full_stats["Carries"] == 0].isnull().sum()

Rank                        0
PlayerID                    0
Name                        0
Team                        0
Position                    0
Played                      0
RushingAttempts             0
RushingYards                0
RushingYardsPerAttempt      0
RushingTouchdowns           0
ReceivingTargets            0
Receptions                  0
ReceivingYards              0
ReceivingTouchdowns         0
Fumbles                     0
FumblesLost                 0
FantasyPointsPerGame        0
FantasyPoints               0
Year                        0
PlayerShortName             0
PlayerAgeExact              0
Carries                     0
YardsPerCarry               0
Targets                     0
ReceivingTDs                0
SnapShare                   0
OpportunityShare            0
ShotgunCarryRate            0
ShotgunYardsPerCarry        0
UnderCenterCarryRate        0
UnderCenterYardsPerCarry    0
AverageDefendersInTheBox    0
StackedFrontCarryRate       0
BaseFrontC

In [36]:
qb_full_stats.isnull().sum()

Rank                             0
PlayerID                         0
Name                             0
Team                             0
Position                         0
Played                           0
PassingCompletions               0
PassingAttempts                  0
PassingCompletionPercentage      0
PassingYards                     0
PassingYardsPerAttempt           0
PassingTouchdowns                0
PassingInterceptions             0
PassingRating                    0
RushingAttempts                  0
RushingYards                     0
RushingYardsPerAttempt           0
RushingTouchdowns                0
FantasyPointsPerGame             0
FantasyPoints                    0
Year                             0
PlayerShortName                  0
PlayerAgeExact                   0
PassAttemptsPerGame              0
CompletionPercentage             0
MoneyThrows                      0
TouchdownRate                    0
ProtectionRate                   0
PressuredCompletionP

In [37]:
qb_full_stats.head()

,Rank,PlayerID,Name,Team,Position,Played,PassingCompletions,PassingAttempts,PassingCompletionPercentage,PassingYards,PassingYardsPerAttempt,PassingTouchdowns,PassingInterceptions,PassingRating,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,FantasyPointsPerGame,FantasyPoints,Year,PlayerShortName,PlayerAgeExact,PassAttemptsPerGame,CompletionPercentage,MoneyThrows,TouchdownRate,ProtectionRate,PressuredCompletionPercentage,AirYards,AirYardsPerGame,AirYardsPerAttempt,DeepBallAttempts,DeepBallCompletionPercentage
0,1,19801,Josh Allen,BUF,QB,17,409,646,63.3,4407,6.8,36,15,92.17,122,763,6.3,6,23.7,402.58,2021,J. Allen,26.1,37.9,63.41,29,5.58,77.94,37.83,2785.0,163.82,4.31,80,42.50
1,2,21681,Justin Herbert,LAC,QB,17,443,672,65.9,5014,7.5,38,15,97.66,63,302,4.8,3,22.4,380.76,2021,J. Herbert,24.3,39.5,65.92,30,5.65,88.62,41.34,2678.0,157.52,3.98,71,43.66
2,3,4314,Tom Brady,TB,QB,17,485,719,67.5,5316,7.4,43,12,102.08,28,81,2.9,2,22.0,374.74,2021,T. Brady,44.9,42.3,67.45,44,5.98,92.98,46.84,2792.0,164.23,3.88,87,40.22
3,4,18890,Patrick Mahomes,KC,QB,17,436,658,66.3,4839,7.4,37,13,98.45,66,381,5.8,2,21.3,361.66,2021,P. Mahomes,26.8,38.7,66.26,32,5.62,85.27,40.65,2219.0,130.52,3.37,79,36.70
4,5,2593,Aaron Rodgers,GB,QB,16,366,531,68.9,4115,7.7,37,4,111.90,33,101,3.1,3,20.8,332.30,2021,A. Rodgers,38.6,33.2,68.92,37,6.96,88.94,39.75,1956.0,122.25,3.68,69,39.13


In [38]:
rb_full_stats.head()

,Rank,PlayerID,Name,Team,Position,Played,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGame,FantasyPoints,Year,PlayerShortName,PlayerAgeExact,Carries,YardsPerCarry,Targets,ReceivingTDs,SnapShare,OpportunityShare,ShotgunCarryRate,ShotgunYardsPerCarry,UnderCenterCarryRate,UnderCenterYardsPerCarry,AverageDefendersInTheBox,StackedFrontCarryRate,BaseFrontCarryRate,LightFrontCarryRate
0,1,21682,Jonathan Taylor,IND,RB,17,332,1811,5.5,18,51,40,360,2,4,2,19.6,333.1,2021,J. Taylor,23.5,332,5.45,53,2,68.56,70.77,37.65,1.0,62.34,1.0,6.07,13.25,28.31,57.83
1,2,19562,Austin Ekeler,LAC,RB,16,206,911,4.4,12,94,70,647,8,4,3,17.1,273.8,2021,A. Ekeler,27.1,206,4.42,94,8,67.10,67.41,34.46,1.0,65.53,1.0,6.65,19.90,37.37,41.74
2,3,18858,Joe Mixon,CIN,RB,16,292,1205,4.1,13,48,42,314,3,2,1,15.4,245.9,2021,J. Mixon,26.0,292,4.12,48,3,67.57,74.88,25.34,1.0,74.65,1.0,6.69,17.80,34.93,47.26
3,4,21768,Najee Harris,PIT,RB,17,307,1200,3.9,7,94,74,467,3,0,0,13.3,226.7,2021,N. Harris,24.3,307,3.90,94,3,84.45,86.42,52.44,1.0,46.57,1.0,6.52,16.93,29.64,50.81
4,5,18983,James Conner,ARI,RB,15,202,752,3.7,15,39,37,375,3,2,0,14.7,220.7,2021,J. Conner,27.2,202,3.72,39,3,57.93,58.92,84.65,1.0,15.34,1.0,6.29,12.87,21.78,64.35


In [39]:
te_full_stats.head()

,Rank,PlayerID,Name,Team,Position,Played,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,ReceivingLong,ReceivingYardsPerTarget,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGame,FantasyPoints,Year,PlayerShortName,Targets,SnapShare,AverageTargetDistance,AirYards,AirYardsPerGame,AirYardsPerReception,RedZoneTargets,EndzoneTargets,TargetAccuracy,TargetShare,FantasyPointsPerTarget
0,1,19803,Mark Andrews,BAL,TE,17,154,107,69.5,1361,9,43,8.8,12.7,1,0,0.0,0,1,0,11.4,194.1,2021,M. Andrews,154,76.54,9.35,1440.0,84.70,13.5,20,13,7.35,26.64,1.96
1,2,15048,Travis Kelce,KC,TE,16,134,92,68.7,1125,9,69,8.4,12.2,2,3,1.5,1,1,1,10.7,170.8,2021,T. Kelce,135,83.09,7.19,971.0,60.68,10.6,16,4,7.74,22.31,1.95
2,3,19920,Dalton Schultz,DAL,TE,17,104,78,75.0,808,8,32,7.8,10.4,0,0,0.0,0,0,0,7.7,130.8,2021,D. Schultz,104,82.59,7.29,759.0,44.64,9.7,14,7,8.17,16.35,2.01
3,4,19063,George Kittle,SF,TE,14,94,71,75.5,910,6,48,9.7,12.8,3,20,6.7,0,2,1,9.1,127.0,2021,G. Kittle,95,92.35,6.70,637.0,45.50,9.0,8,5,7.68,24.86,2.08
4,5,10974,Rob Gronkowski,TB,TE,12,89,55,61.8,802,6,42,9.0,14.6,0,0,0.0,0,0,0,9.7,116.2,2021,R. Gronkowski,89,76.19,10.53,938.0,78.16,17.1,12,8,7.69,17.21,1.92


In [40]:
wr_full_stats.head()

,Rank,PlayerID,Name,Team,Position,Played,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,ReceivingLong,ReceivingYardsPerTarget,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGame,FantasyPoints,Year,PlayerShortName,Targets,ReceivingTDs,SnapShare,AverageTargetDistance,AirYards,AirYardsPerGame,AirYardsPerReception,RedZoneTargets,EndzoneTargets,TargetAccuracy,TargetShare,FantasyPointsPerTarget
0,1,18882,Cooper Kupp,LAR,WR,17,191,145,75.9,1947,16,59,10.2,13.4,4,18,4.5,0,0,0,17.3,294.50,2021,C. Kupp,191,16,94.01,8.32,1590.0,93.52,11.0,38,11,7.76,31.72,2.30
1,2,20932,Deebo Samuel,SF,WR,16,121,77,63.6,1405,6,83,11.6,18.2,59,365,6.2,8,4,2,16.4,261.96,2021,D. Samuel,120,6,81.44,8.09,971.0,60.68,12.6,10,6,7.14,27.77,2.82
2,3,22564,Ja'Marr Chase,CIN,WR,17,128,81,63.3,1455,13,82,11.4,18.0,7,21,3.0,0,2,1,13.2,223.60,2021,J. Chase,128,13,86.11,11.96,1532.0,90.11,18.9,12,12,7.30,23.70,2.38
3,4,21685,Justin Jefferson,MIN,WR,17,167,108,64.7,1616,10,56,9.7,15.0,6,14,2.3,0,1,1,13.1,222.40,2021,J. Jefferson,167,10,89.01,11.95,1996.0,117.41,18.5,23,15,7.41,29.87,1.98
4,5,16470,Davante Adams,GB,WR,16,169,123,72.8,1553,11,59,9.2,12.6,0,0,0.0,0,0,0,13.8,221.30,2021,D. Adams,169,11,80.33,8.99,1520.0,95.00,12.4,28,11,7.33,31.58,2.04


In [41]:
#Hand picked duplicate values. Columns aren't identical, but close enough
wr_dups = ["Targets", "ReceivingTDs"]
rb_dups = ["Carries", "RushingYardsPerAttempt", "ReceivingTargets", "ReceivingTDs"]
te_dups = ["Targets"]
qb_dups = ["PassingCompletionPercentage"]

In [42]:
# # Drop redundant columns from previous line
# qb_full_stats.drop(columns = qb_dups, axis= 1, inplace= True)
# rb_full_stats.drop(columns = rb_dups, axis= 1, inplace= True)
# te_full_stats.drop(columns = te_dups, axis= 1, inplace= True)
# wr_full_stats.drop(columns = wr_dups, axis= 1, inplace= True)

In [43]:
#Check more null columns for RBs
rb_full_stats.isnull().sum()

Rank                         0
PlayerID                     0
Name                         0
Team                         0
Position                     0
Played                       0
RushingAttempts              0
RushingYards                 0
RushingYardsPerAttempt       0
RushingTouchdowns            0
ReceivingTargets             0
Receptions                   0
ReceivingYards               0
ReceivingTouchdowns          0
Fumbles                      0
FumblesLost                  0
FantasyPointsPerGame         0
FantasyPoints                0
Year                         0
PlayerShortName              0
PlayerAgeExact               0
Carries                      0
YardsPerCarry                6
Targets                      0
ReceivingTDs                 0
SnapShare                    0
OpportunityShare             6
ShotgunCarryRate             6
ShotgunYardsPerCarry        43
UnderCenterCarryRate         6
UnderCenterYardsPerCarry    16
AverageDefendersInTheBox     6
StackedF

In [56]:
rb_full_stats["YardsPerCarry"].map(rb_full_stats["YardsPerCarry"].copy(),na_action = rb_full_stats["RushingYardsPerAttempt"] )

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
30      NaN
31      NaN
32      NaN
33      NaN
34      NaN
35      NaN
36      NaN
37      NaN
38      NaN
39      NaN
40      NaN
41      NaN
42      NaN
43      NaN
44      NaN
45      NaN
46      NaN
47      NaN
48      NaN
49      NaN
50      NaN
51      NaN
52      NaN
53      NaN
54      NaN
55      NaN
56      NaN
57      NaN
58      NaN
59      NaN
60      NaN
61      NaN
62      NaN
63      NaN
64      NaN
65      NaN
66      NaN
67      NaN
68      NaN
69      NaN
70      NaN
71      NaN
72      NaN
73      NaN
74      NaN
75      NaN
76      NaN
77      NaN
78      NaN
79      NaN
80      NaN
81      NaN
82      NaN
83  

In [45]:
rb_full_stats[rb_full_stats["ShotgunYardsPerCarry"].isnull()]


,Rank,PlayerID,Name,Team,Position,Played,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGame,FantasyPoints,Year,PlayerShortName,PlayerAgeExact,Carries,YardsPerCarry,Targets,ReceivingTDs,SnapShare,OpportunityShare,ShotgunCarryRate,ShotgunYardsPerCarry,UnderCenterCarryRate,UnderCenterYardsPerCarry,AverageDefendersInTheBox,StackedFrontCarryRate,BaseFrontCarryRate,LightFrontCarryRate
234,73,15150,Cordarrelle Patterson,CHI,RB,16,64,232,3.6,1,25,21,132,0,0,0,2.6,42.4,2020,C. Patterson,31.3,64,NaN,25,0,18.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,100,20819,Alec Ingold,LV,FB,16,3,4,1.3,0,17,12,110,1,0,0,1.1,17.4,2020,A. Ingold,26.0,3,1.3,17,1,19.30,5.7,0.0,NaN,100.0,1.3,7.67,33.3,66.7,0.0
266,106,22310,Gabe Nabers,LAC,FB,16,2,7,3.5,0,7,5,25,2,0,0,1.0,15.2,2020,G. Nabers,24.7,2,3.5,7,2,14.50,3.3,0.0,NaN,100.0,3.5,7.50,50.0,50.0,0.0
285,128,13418,Anthony Sherman,KC,FB,13,3,8,2.7,0,2,1,5,1,0,0,0.6,7.3,2020,A. Sherman,33.6,3,2.7,2,1,11.40,4.0,0.0,NaN,100.0,2.7,8.00,66.7,33.3,0.0
292,137,16928,Michael Burton,NO,FB,15,7,18,2.6,0,4,4,28,0,0,0,0.3,4.6,2020,M. Burton,30.4,7,2.6,4,0,21.90,5.1,0.0,NaN,100.0,2.6,7.43,57.1,28.6,14.3
299,144,19072,Alex Armah,CAR,FB,16,6,9,1.5,0,9,5,18,0,0,0,0.2,2.7,2020,A. Armah,28.1,6,1.5,9,0,13.90,5.0,0.0,NaN,100.0,1.5,7.83,83.3,16.7,0.0
302,147,18093,Andy Janovich,CLE,FB,14,2,4,2.0,0,5,2,13,0,0,0,0.1,1.7,2020,A. Janovich,29.1,2,2.0,5,0,16.80,3.2,0.0,NaN,100.0,2.0,7.50,50.0,50.0,0.0
391,63,18464,J.D. McKissic,DET,RB,16,38,205,5.4,0,42,34,233,1,0,0,3.1,49.8,2019,J. McKissic,28.9,38,NaN,42,1,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,91,18652,C.J. Ham,MIN,FB,16,7,17,2.4,0,26,17,149,1,1,1,1.3,20.6,2019,C. Ham,29.0,7,2.4,27,1,22.50,7.6,0.0,NaN,114.3,2.0,8.29,42.9,42.9,28.6
421,93,15150,Cordarrelle Patterson,CHI,RB,16,17,103,6.1,0,17,11,83,0,0,0,1.2,18.6,2019,C. Patterson,31.3,17,NaN,17,0,19.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
qb_column_order = ["PlayerID","Name","PlayerShortName","PlayerAgeExact","Rank","Team","Position","Played","Year","PassingCompletions","PassingAttempts","PassingCompletionPercentage","PassingYards","PassingYardsPerAttempt","PassingTouchdowns","PassingInterceptions","PassingRating","PassAttemptsPerGame","CompletionPercentage","PressuredCompletionPercentage","AirYards","AirYardsPerGame","AirYardsPerAttempt","DeepBallAttempts","DeepBallCompletionPercentage","MoneyThrows","RushingAttempts","RushingYards","RushingYardsPerAttempt","RushingTouchdowns","TouchdownRate","ProtectionRate","FantasyPointsPerGame","FantasyPoints"]
rb_column_order = ["PlayerID","Name","PlayerShortName","PlayerAgeExact","Rank","Team","Position","Played","Year","RushingAttempts","RushingYards","RushingYardsPerAttempt","RushingTouchdowns","Targets","Receptions","ReceivingYards","ReceivingTouchdowns","Fumbles","FumblesLost","SnapShare","FantasyPointsPerGame","FantasyPoints"]
wr_column_order = ["PlayerID","Name","PlayerShortName","Rank","Team","Position","Played","Year","ReceivingTargets","Receptions","ReceptionPercentage","ReceivingYards","ReceivingLong","ReceivingYardsPerTarget","ReceivingYardsPerReception","AverageTargetDistance","AirYards","AirYardsPerGame","AirYardsPerReception","RedZoneTargets","EndzoneTargets","TargetAccuracy","TargetShare","RushingAttempts","RushingYards","RushingYardsPerAttempt","RushingTouchdowns","Fumbles","FumblesLost","SnapShare","FantasyPointsPerTarget","FantasyPointsPerGame","FantasyPoints"]
te_column_order = ["PlayerID","Name","PlayerShortName","Rank","Team","Position","Played","Year","ReceivingTargets","Receptions","ReceptionPercentage","ReceivingYards","ReceivingTouchdowns","ReceivingLong","ReceivingYardsPerTarget","ReceivingYardsPerReception","AverageTargetDistance","AirYards","AirYardsPerGame","AirYardsPerReception","RedZoneTargets","EndzoneTargets","TargetAccuracy","TargetShare","RushingAttempts","RushingYards","RushingYardsPerAttempt","RushingTouchdowns","Fumbles","FumblesLost","SnapShare","FantasyPointsPerTarget","FantasyPointsPerGame","FantasyPoints"]


In [47]:
#qb_full_stats = qb_full_stats[qb_column_order]
# wr_full_stats = wr_full_stats[wr_column_order]
# rb_full_stats = rb_full_stats[rb_column_order]
# te_full_stats = te_full_stats[te_column_order]

In [48]:
# qb_full_stats.to_sql("QB_Full_Stats", con, if_exists="replace")
# wr_full_stats.to_sql("WR_Full_Stats", con, if_exists="replace")
# rb_full_stats.to_sql("RB_Full_Stats", con, if_exists="replace")
# te_full_stats.to_sql("TE_Full_Stats", con, if_exists="replace")


In [49]:
# Getting all tables from sqlite_master
# sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""

In [50]:
# # executing our sql query
# cursor.execute(sql_query)
# print("List of tables\n")
    
# # printing all tables list
# print(cursor.fetchall())

In [51]:
# con.close()